In [1]:
#!pip install html5lib

In [2]:
#!pip install -U selenium

In [28]:
#!pip install pyautogui

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains # для имитации кликов
from selenium.webdriver.support.ui import WebDriverWait # для поиска капчи и имитации кликов
from selenium.webdriver.support import expected_conditions as EC # для условий
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
import re
from playwright.sync_api import sync_playwright
from playwright.async_api import async_playwright
from selenium.webdriver.chrome.options import Options
import keyboard
import undetected_chromedriver as uc
import pyautogui

In [4]:
df = pd.read_csv('sample_submission.csv')
display(df.head())
print(df.info())

,sell_id,price
0,1100575026,0
1,1100549428,0
2,1100658222,0
3,1100937408,0
4,1101037972,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   sell_id  34686 non-null  int64
 1   price    34686 non-null  int64
dtypes: int64(2)
memory usage: 542.1 KB
None


In [5]:
df1 = pd.read_csv('test.csv')
display(df1.head())
print(df1.info())

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,...,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня
0,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,синий,NaN,"Все автомобили, представленные в продаже, прох...",1.2 LTR,105 N12,"{""engine-proof"":true,""tinted-glass"":true,""airb...",бензин,...,LIFTBACK ROBOT 1.2,роботизированная,EUROPEAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
1,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,чёрный,NaN,ЛОТ: 01217195\nАвтопрага Север\nДанный автомоб...,1.6 LTR,110 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK MECHANICAL 1.6,механическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
2,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/superb/11...,серый,"{""id"":""20026336"",""name"":""Ambition"",""available_...","Все автомобили, представленные в продаже, прох...",1.8 LTR,152 N12,"{""cruise-control"":true,""tinted-glass"":true,""es...",бензин,...,LIFTBACK ROBOT 1.8,роботизированная,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
3,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,коричневый,"{""id"":""20803582"",""name"":""Ambition"",""available_...",КОМПЛЕКТ ЗИМНЕЙ (ЛЕТНЕЙ) РЕЗИНЫ ПО СЕЗОНУ В ПО...,1.6 LTR,110 N12,"{""cruise-control"":true,""roller-blind-for-rear-...",бензин,...,LIFTBACK AUTOMATIC 1.6,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен
4,лифтбек,SKODA,https://auto.ru/cars/used/sale/skoda/octavia/1...,белый,NaN,ЛОТ: 01220889\nАвтопрага Север\n\nВы можете по...,1.8 LTR,152 N12,"{""cruise-control"":true,""asr"":true,""esp"":true,""...",бензин,...,LIFTBACK AUTOMATIC 1.8,автоматическая,EUROPEAN,1 владелец,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34686 entries, 0 to 34685
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   bodyType              34686 non-null  object
 1   brand                 34686 non-null  object
 2   car_url               34686 non-null  object
 3   color                 34686 non-null  object
 4   complectation_dict    6418 non-null   object
 5   description           34686 non-null  object
 6   engineDisplacement    34686 non-null  object
 7   enginePower           34686 non-null  object
 8   equipment_dict        24690 non-null  object
 9   fuelType              34686 non-null  object
 10  image                 34686 non-null  object
 11  mileage               34686 non-null  int64 
 12  modelDate             34686 non-null  int64 
 13  model_info            34686 non-null  object
 14  model_name            34686 non-null  object
 15  name                  34686 non-null

In [13]:
from time import sleep
from urllib.parse import unquote
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement
import pathes

TABLE_COLUMNS = ['Название', 'Телефон', 'Адрес', 'Ссылка']
TABLE = {column: [] for column in TABLE_COLUMNS}


def get_element_text(driver: WebDriver, path: str) -> str:
    try:
        return driver.find_element(By.XPATH, path).text
    except NoSuchElementException:
        return ''


def move_to_element(driver: WebDriver, element: WebElement | WebDriver) -> None:
    try:
        webdriver.ActionChains(driver).move_to_element(element).perform()
    except StaleElementReferenceException:
        pass


def element_click(driver: WebDriver | WebElement, path: str) -> bool:
    try:
        driver.find_element(By.XPATH, path).click()
        return True
    except:
        return False


def main():
    search_query = 'Пиццерия'
    url = f'https://2gis.ru/novorossiysk/search/{search_query}'
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(url)
    element_click(driver, pathes.main_banner)
    element_click(driver, pathes.cookie_banner)
    count_all_items = int(get_element_text(driver, pathes.items_count))
    pages = round(count_all_items / 12 + 0.5)
    for _ in range(pages):
        main_block = driver.find_element(By.XPATH, pathes.main_block)
        count_items = len(main_block.find_elements(By.XPATH, 'div'))
        for item in range(1, count_items + 1):
            if main_block.find_element(By.XPATH, f'div[{item}]').get_attribute('class'):
                continue
            item_clicked = element_click(main_block, f'div[{item}]/div/div[2]')
            if not item_clicked:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                element_click(main_block, f'div[{item}]/div/div[2]')
            title = get_element_text(driver, pathes.title)
            phone_btn_clicked = element_click(driver, pathes.phone_btn)
            phone = get_element_text(driver, pathes.phone) if phone_btn_clicked else ''
            move_to_element(driver, main_block)
            link = unquote(driver.current_url)
            address = get_element_text(driver, pathes.address)
            TABLE['Название'].append(title)
            TABLE['Телефон'].append(phone)
            TABLE['Адрес'].append(address)
            TABLE['Ссылка'].append(link)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        element_click(driver, pathes.next_page_btn)
        sleep(0.5)
    driver.quit()
    pd.DataFrame(TABLE).to_excel(f"{search_query}.xlsx")


if __name__ == '__main__':
    main()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.112)
Stacktrace:
	GetHandleVerifier [0x00007FF60608AD32+56930]
	(No symbol) [0x00007FF605FFF632]
	(No symbol) [0x00007FF605EB42E5]
	(No symbol) [0x00007FF605E91D4C]
	(No symbol) [0x00007FF605F223F7]
	(No symbol) [0x00007FF605F37891]
	(No symbol) [0x00007FF605F1BA43]
	(No symbol) [0x00007FF605EED438]
	(No symbol) [0x00007FF605EEE4D1]
	GetHandleVerifier [0x00007FF606406ABD+3709933]
	GetHandleVerifier [0x00007FF60645FFFD+4075821]
	GetHandleVerifier [0x00007FF60645818F+4043455]
	GetHandleVerifier [0x00007FF606129766+706710]
	(No symbol) [0x00007FF60600B90F]
	(No symbol) [0x00007FF606006AF4]
	(No symbol) [0x00007FF606006C4C]
	(No symbol) [0x00007FF605FF6904]
	BaseThreadInitThunk [0x00007FFEA26D7344+20]
	RtlUserThreadStart [0x00007FFEA39426B1+33]


In [8]:
df = pd.read_excel('Пиццерия.xlsx')
print(df.shape)
display(df.head())

(112, 5)


,Unnamed: 0,Название,Телефон,Адрес,Ссылка
0,0,ПРОСУШИ,8‒800‒775‒55‒99,"Молодёжная, 8Б1 этаж2 филиала\n14-й м-н, Южный...",https://2gis.ru/novorossiysk/search/Пиццерия
1,1,Додо Пицца,8‒8003020060,"Энгельса, 50цокольный этаж8 филиалов\nЦентраль...",https://2gis.ru/novorossiysk/search/Пиццерия/f...
2,2,Troller Brewing,+7‒988‒311‒00‒20,"Борисовская, 21 этаж; вход с ул. Видова6 филиа...",https://2gis.ru/novorossiysk/search/Пиццерия/f...
3,3,Легенды,+7‒999‒639‒60‒06,"Островского, 85\nГеленджик, Краснодарский край...",https://2gis.ru/novorossiysk/search/Пиццерия/f...
4,4,Папа Джонс,+7 (86141) 5‒81‒81,"Советская, 21\nГеленджик, Краснодарский край, ...",https://2gis.ru/gelendzhik/search/Пиццерия/fir...


In [12]:
class OzonSellerParse:
    def __init__(self, keyword: str):
        self.keyword = keyword

    def __page_down(self, page):
        page.evaluate('''
                                const scrollStep = 200; // Размер шага прокрутки (в пикселях)
                                const scrollInterval = 100; // Интервал между шагами (в миллисекундах)

                                const scrollHeight = document.documentElement.scrollHeight;
                                let currentPosition = 0;
                                const interval = setInterval(() => {
                                    window.scrollBy(0, scrollStep);
                                    currentPosition += scrollStep;

                                    if (currentPosition >= scrollHeight) {
                                        clearInterval(interval);
                                    }
                                }, scrollInterval);
                            ''')

    def __get_seller_name(self, url: str):
        self.page2 = self.context.new_page()
        self.page2.goto(url=url)

        self.__page_down(self.page2)
        try:
            self.page2.wait_for_selector(f':text("Продавец")')
            elements = self.page2.query_selector_all('a[href^="https://www.ozon.ru/sell"]')
            seller_name = elements[1].inner_text()
            print(seller_name)
        except Exception:
            return

    def __get_links(self):
        self.page.wait_for_selector("#paginatorContent")
        self.__page_down(self.page)
        self.page.wait_for_selector(f':text("Дальше")')

        search_result = self.page.query_selector("#paginatorContent")
        links = search_result.query_selector_all(".tile-hover-target")
        print(len(links))
        for count, link in enumerate(links):
            if count > 5: break
            url = "https://ozon.by" + link.get_attribute('href')
            self.__get_seller_name(url=url)

    def parse(self):
        with sync_playwright() as playwright:
            browser = playwright.chromium.launch(headless=False)
            self.context = browser.new_context()
            self.page = self.context.new_page()
            self.page.goto("https://ozon.by/")
            self.page.get_by_placeholder("Искать на Ozon").type(text=self.keyword, delay=0.3)
            self.page.query_selector("button[type='submit']").click()
            self.__get_links()


if __name__ == "__main__":
    OzonSellerParse("вентилятор").parse()
    time.sleep(20)

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.

In [47]:
url = 'https://auto.ru/cars/2024-year/used/?price_from=1&price_to=5000000&sort=price-desc'

TABLE_COLUMNS = ['Название', 'Телефон', 'Адрес', 'Ссылка']
TABLE = {column: [] for column in TABLE_COLUMNS}

try:
    options = webdriver.ChromeOptions() 
    #options.add_argument("start-maximized")
    options.add_argument('disable-infobars')
    driver = uc.Chrome(executable_path=r'C:\WebDrivers\chromedriver.exe', chrome_options=options)
    driver.get(url) 
    # проходим до ячйки капчи с  подтверждением нажатиями кнопок клавиатуры
    for i in range(5):
        keyboard. press('Tab')
        time.sleep(0.4)
    # нажатие кнопки капчи
    keyboard. press('Enter')
    # ожидаем загрузки страницы
    driver.implicitly_wait(10)
except:
    pass
finally:   
    # ищем количество объявлений по запросу
    all_items = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[3]/div[2]/section/div[2]/div/div[2]/div[4]/div[3]/button/span/span").text
    count_all_items = int(re.search('\d+', all_items)[0])
    # пауза
    driver.implicitly_wait(10)
    pages = round(count_all_items / 37+1)
    print(count_all_items, pages)
       
    main_block = driver.find_elements(By., "/html/body/div[1]/div/div[2]/div[3]/div[2]/section/div[2]/div/div[7]/div[2]/div/div[2]/div[1]/h3/a")
    count_items = len(main_block)
    print(count_items)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    main_block[0].click()
    driver.implicitly_wait(10)
    print(driver.current_url)
    pyautogui.hotkey('ctrl', 'w')
    driver.implicitly_wait(10)
    main_block[1].click()
    driver.implicitly_wait(10)
    print(driver.current_url)
   
    #pyautogui.hotkey('ctrl', 'w')
    time.sleep(2)
    #driver.close()
    driver.quit() #close()# закрыть 1 вкладку


485 14
0


IndexError: list index out of range

In [51]:
for _ in range(pages):
        # считаем количество объявлений на странице
        main_block = driver.find_elements(By.CLASS_NAME, "ListingItem__summary")
        count_items = len(main_block)
        print(count_items)
    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        main_block[0].click()
        driver.implicitly_wait(10)
        driver.close()
        
        # проматываем страницы
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.CONTROL + Keys.ARROW_RIGHT)
        time.sleep(2)




for item in range(count_items + 1):
            item_clicked = element_click(main_block, f'div[{item}]/div/div[2]')
            if not item_clicked:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                driver.find_elements(By.XPATH, '/html/body/div[8]/div/button/span/span')
                element_click(main_block, f'div[{item}]/div/div[2]')
            title = get_element_text(driver, pathes.title)
            phone_btn_clicked = element_click(driver, pathes.phone_btn)
            phone = get_element_text(driver, pathes.phone) if phone_btn_clicked else ''
            move_to_element(driver, main_block)
            link = unquote(driver.current_url)
            address = get_element_text(driver, pathes.address)
            TABLE['Название'].append(title)
            TABLE['Телефон'].append(phone)
            TABLE['Адрес'].append(address)
            TABLE['Ссылка'].append(link)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        element_click(driver, pathes.next_page_btn)


url = 'https://auto.ru/cars/2024-year/used/?price_from=1&price_to=5000000&sort=price-desc'
import undetected_chromedriver as uc
#driver = uc.Chrome(headless=False,use_subprocess=False)

TABLE_COLUMNS = ['Название', 'Телефон', 'Адрес', 'Ссылка']
TABLE = {column: [] for column in TABLE_COLUMNS}


def get_element_text(driver: WebDriver, path: str) -> str:
    try:
        return driver.find_element(By.XPATH, path).text
    except NoSuchElementException:
        return ''


def move_to_element(driver: WebDriver, element: WebElement | WebDriver) -> None:
    try:
        webdriver.ActionChains(driver).move_to_element(element).perform()
    except StaleElementReferenceException:
        pass


def element_click(driver: WebDriver | WebElement, path: str) -> bool:
    try:
        driver.find_element(By.XPATH, path).click()
        return True
    except:
        return False

try:
    options = webdriver.ChromeOptions() 
    options.add_argument("start-maximized")
    options.add_argument('disable-infobars')
    driver = uc.Chrome(executable_path=r'C:\WebDrivers\chromedriver.exe', chrome_options=options)
    driver.get(url) 
    for i in range(5):
        keyboard. press('Tab')
        time.sleep(0.3)
    # нажатие кнопки капчи
    keyboard. press('Enter')
    # ожидаем загрузки страницы
    driver.implicitly_wait(10)
except:
    pass
    
finally:
    def main():
    count_all_items = int(get_element_text(driver, pathes.items_count))
    pages = round(count_all_items / 12 + 0.5)
    for _ in range(pages):
        main_block = driver.find_element(By.XPATH, pathes.main_block)
        count_items = len(main_block.find_elements(By.XPATH, 'div'))
        for item in range(1, count_items + 1):
            if main_block.find_element(By.XPATH, f'div[{item}]').get_attribute('class'):
                continue
            item_clicked = element_click(main_block, f'div[{item}]/div/div[2]')
            if not item_clicked:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                element_click(main_block, f'div[{item}]/div/div[2]')
            title = get_element_text(driver, pathes.title)
            phone_btn_clicked = element_click(driver, pathes.phone_btn)
            phone = get_element_text(driver, pathes.phone) if phone_btn_clicked else ''
            move_to_element(driver, main_block)
            link = unquote(driver.current_url)
            address = get_element_text(driver, pathes.address)
            TABLE['Название'].append(title)
            TABLE['Телефон'].append(phone)
            TABLE['Адрес'].append(address)
            TABLE['Ссылка'].append(link)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        element_click(driver, pathes.next_page_btn)
        sleep(0.5)
    driver.quit()
    pd.DataFrame(TABLE).to_excel(f"{search_query}.xlsx")


    my_elem = driver.find_elements(By.CLASS_NAME, "ListingItem__summary").text()
    print(my_elem)
    my_elem[0].click()
    driver.implicitly_wait(10)
    driver.switch_to.window(driver.window_handles[1])
    print(driver.current_url)
    
    #for my_elem in driver.find_elements(By.CLASS_NAME, "ListingItem__summary"):
        #my_elem.click()
        #driver.implicitly_wait(10)
        #time.sleep(np.random.uniform(2, 3))
        #print(driver.current_url)
        #driver.switch_to.window(driver.window_handles[1])
        #print(driver.current_url)
        #driver.close(driver.window_handles[1])
        #driver.implicitly_wait(10)
        #time.sleep(np.random.uniform(2, 3))
         #if not item_clicked:
                #driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                #element_click(main_block, f'div[{item}]/div/div[2]')
    driver.close()
    driver.quit() #close()# закрыть 1 вкладку
    
if __name__ == '__main__':
    main()

AttributeError: 'list' object has no attribute 'text'

In [20]:
df = pd.DataFrame({'price': [], 'status': [], 'generation': [], 'age': [], 'run': [], 'body': []})
url = 'https://auto.ru/cars/used/sale/vaz/vesta/1122095801-58c0b7cb/'
driver = webdriver.Firefox()
driver.get(url)
try:   
    wait = WebDriverWait(driver, 3)
    # класс кнопки капчи
    button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "CheckboxCaptcha-Checkbox")))
    # нажатие кнопки капчи
    button.click()
except:
    # ожидаем загрузки страницы
    time.sleep(np.random.uniform(1, 2))
finally:
    time.sleep(np.random.uniform(2, 3))
    e_price = driver.find_element(By.CLASS_NAME , "OfferPriceCaption__price")
    e_info = driver.find_elements(By.CLASS_NAME , "CardInfoRow__cell")
    df.loc[len(df.index )] = [e_price.text, e_info[1].text, e_info[3].text, e_info[5].text, e_info[7].text, e_info[9].text]
    driver.quit() #close()# закрыть 1 вкладку

NoSuchElementException: Message: Unable to locate element: .OfferPriceCaption__price; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:509:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16


In [ ]:
b = re.search('.*', a[0])[0]
print(b)
print(a[0])

In [ ]:
print(df)

In [ ]:
df.info()